In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Define base path and folder paths
base_path = '/content/drive/My Drive/Colab Notebooks'

# Corrected folder paths
no_rust_train_path = os.path.join(base_path, 'Corrosion_Train', 'no rust')
rust_train_path = os.path.join(base_path, 'Corrosion_Train', 'rust')
no_rust_test_path = os.path.join(base_path, 'No Rust_Test')
rust_test_path = os.path.join(base_path, 'Rust_Test')

In [3]:
from PIL import Image
import glob

# Function to load images from a directory
def load_images_from_folder(folder_path):
    images = []
    for filename in glob.glob(os.path.join(folder_path, '*')):
        # Check if it's a file (not a directory) and has an image extension
        if os.path.isfile(filename) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                img = Image.open(filename)
                images.append(img)
            except Exception as e:
                print(f"Could not load image {filename}: {e}")
    return images


# Load images from each folder
no_rust_train_images = load_images_from_folder(no_rust_train_path)
rust_train_images = load_images_from_folder(rust_train_path)
no_rust_test_images = load_images_from_folder(no_rust_test_path)
rust_test_images = load_images_from_folder(rust_test_path)


# Print the number of images loaded from each folder
print(f"Number of images in no rust train: {len(no_rust_train_images)}")
print(f"Number of images in rust train: {len(rust_train_images)}")
print(f"Number of images in No Rust Test: {len(no_rust_test_images)}")
print(f"Number of images in Rust Test: {len(rust_test_images)}")

Number of images in no rust train: 405
Number of images in rust train: 345
Number of images in No Rust Test: 10
Number of images in Rust Test: 10


In [4]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2


def preprocess_images(images, target_size):
    processed_images = []
    for img in images:
        if isinstance(img, np.ndarray):
            # Convert numpy array to uint8 if necessary
            img = img.astype(np.uint8)

            # Resize using OpenCV
            img = cv2.resize(img, target_size)
        else:
            # Ensure the image is in RGB format
            img = img.convert('RGB')
            img = img.resize(target_size)  # Resize image to target size

        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        processed_images.append(img_array)

    return np.array(processed_images)



# Define target image size and preprocess images
target_size = (128, 128)  # Adjust to your image size
no_rust_train_images = preprocess_images(no_rust_train_images, target_size)
rust_train_images = preprocess_images(rust_train_images, target_size)

# Combine the images and create labels
X_train = np.concatenate([no_rust_train_images, rust_train_images], axis=0)
y_train = np.concatenate([np.zeros(len(no_rust_train_images)), np.ones(len(rust_train_images))], axis=0)

# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes=2)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical


# Define the variables
num_filters = 32  # Number of filters in the Conv2D layer
filter_size = (3, 3)  # Size of the filter/kernel
pool_size = (2, 2)  # Size of the pooling window

# Define CNN model
model = Sequential([
    Conv2D(num_filters, filter_size, activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes (rust and no rust)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 655ms/step - accuracy: 0.6437 - loss: 4.1219 - val_accuracy: 0.1200 - val_loss: 1.5149
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 553ms/step - accuracy: 0.6568 - loss: 0.6305 - val_accuracy: 0.6467 - val_loss: 0.7245
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 18s 430ms/step - accuracy: 0.7686 - loss: 0.4849 - val_accuracy: 0.5400 - val_loss: 0.8571
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 524ms/step - accuracy: 0.8296 - loss: 0.4006 - val_accuracy: 0.4800 - val_loss: 0.9485
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 584ms/step - accuracy: 0.8273 - loss: 0.3737 - val_accuracy: 0.4600 - val_loss: 0.9883
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 513ms/step - accuracy: 0.8855 - loss: 0.3266 - val_accuracy: 0.6533 - val_loss: 0.7762
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 552ms/step - accuracy: 0.8929 - loss: 0.2733 - val_accuracy: 0.7133 - val_loss: 0.5385
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 604ms/step - accuracy: 0.9237 - loss: 0.2269 - val_accu

In [7]:
# Preprocess test images
no_rust_test_images = preprocess_images(no_rust_test_images, target_size)
rust_test_images = preprocess_images(rust_test_images, target_size)

# Combine test images and create labels
X_test = np.concatenate([no_rust_test_images, rust_test_images], axis=0)
y_test = np.concatenate([np.zeros(len(no_rust_test_images)), np.ones(len(rust_test_images))], axis=0)
y_test = to_categorical(y_test, num_classes=2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.9500 - loss: 0.2870
Test accuracy: 95.00%


# Restnet50

In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

#Define the base model with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # 2 classes (rust and no rust)

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model (ResNet50)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of data for validation
)

# Train the model using data augmentation
batch_size = 16
epochs = 10

train_generator = datagen.flow(X_train, y_train, batch_size=batch_size, subset='training')
validation_generator = datagen.flow(X_train, y_train, batch_size=batch_size, subset='validation')



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=epochs
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy with ResNet50: {test_accuracy * 100:.2f}%")

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5304 - loss: 0.6966 - val_accuracy: 0.0000e+00 - val_loss: 0.7587
Epoch 2/10


AttributeError: 'NoneType' object has no attribute 'items'